In [1]:
# Splitting source xlsx file to smaller chunks for as GitHub repo has file size limit.
import pandas as pd
import numpy as np
# !pip install xlrd 
df = pd.read_excel("CORE_DATA.xlsx")
dfs = np.array_split(df, 5)
for idx in range(len(dfs)):
    dfs[idx].to_pickle("core_data_chunk_"+str(idx)+".pkl")


In [2]:
# load data from chunks
pickles = []
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        if (filename[-3:] == 'pkl'):
            pickles.append(os.path.join(dirname, filename))

df_chunks = []
for pkl in pickles:
    _df = pd.read_pickle(pkl)
    df_chunks.append(_df)

df = pd.concat(df_chunks)
genes = list(df.iloc[:,0])              # set aside gene names
df = df.drop(df.columns[0], axis =1)    # remove all non-numeric data

In [3]:
# import dependencies
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import MinMaxScaler


In [4]:
mat = df.values
scaler = MinMaxScaler()
scaler.fit(mat)
mat_sc = scaler.transform(mat)


In [5]:
kmeans = KMeans(n_clusters=10)
kmeans.fit(mat)
labels = kmeans.labels_
results_km = pd.DataFrame([df.index,labels]).T


In [6]:
hierarchical = AgglomerativeClustering(n_clusters=10, affinity='euclidean', compute_full_tree='auto', linkage='ward', distance_threshold=None).fit(mat)
hierarchical
labels = hierarchical.labels_
results_hc = pd.DataFrame([df.index,labels]).T


In [7]:
df['gene_name'] = pd.Series(genes, index=df.index)
df['k_means_cluster_id'] = results_km.loc[:,1]
df['hierarchical_cluster_id'] = results_hc.loc[:,1]
print(df)

TCGA-02-0047-01A-01R-1849-01  TCGA-02-0055-01A-01R-1849-01  \
0                            0.0000                        0.0000   
1                            3.0732                        0.0000   
2                            6.7611                       15.6973   
3                           54.7036                       31.3945   
4                          886.3210                      885.7744   
...                             ...                           ...   
20526                     3867.9738                    11481.4296   
20527                     1397.0927                      606.0266   
20528                      603.5834                      718.1500   
20529                        6.7611                       12.3336   
20530                        0.0000                        0.0000   

       TCGA-02-2483-01A-01R-1849-01  TCGA-02-2485-01A-01R-1849-01  \
0                            0.0000                        0.0000   
1                            2.9467     